In [ ]:
!pip install git+https://github.com/mvaldenegro/keras-uncertainty.git

from urllib.request import urlretrieve
import requests

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import keras
import keras_uncertainty as ku

  Cloning https://github.com/mvaldenegro/keras-uncertainty.git to /tmp/pip-req-build-so1bp5fb
  Running command git clone --filter=blob:none --quiet https://github.com/mvaldenegro/keras-uncertainty.git /tmp/pip-req-build-so1bp5fb
  Resolved https://github.com/mvaldenegro/keras-uncertainty.git to commit 42f50a36c70003b16b7f343002766708ad2a289a
  Preparing metadata (setup.py) ... done
  Created wheel for Keras-Uncertainty: filename=Keras_Uncertainty-0.0.1-py3-none-any.whl size=37767 sha256=c495431cdf757a3a0c50627ab9ca820fdd1de5680c1770948d5b8243ae437ae9
  Stored in directory: /tmp/pip-ephem-wheel-cache-yzj0zpu6/wheels/ca/4e/2f/97609d4065ffb545a5e867c8b31430161987f4702ecf61a96d
Successfully built Keras-Uncertainty


Keras Uncertainty will use standalone Keras backend

In [ ]:
# Helper function to use throughout the lab
def download_weights(url, destination_filename):
  request = requests.get(url)
  with open(destination_filename, 'wb') as outfile:
      outfile.write(request.content)

## Evaluating the performance of probabilistic models

We first implement two metrics for Bayesian models:

* **Coverage** for regression
* **Brier Score** for classification

Note: in the definitions used below, we will have the following entities:

* $y\in\mathbb{R}^N$ is the vector containing the ground truth.
* $\hat{y}$ is a vector of $M$ probability distributions (the predictive posterior probabilities $q(y|x, \theta)$), $x$ being a set of $N$ data points and $\theta$ the model parameters. We may have a close form of these distributions, or samples obtained, e.g., using Monte-Carlo methods. In the former case, the predictions will be a $M \times N$ matrix which we call $\hat{Y}$.

  We can thus calculate the per-datapoint mean and standard deviations:
  $$
  \mu_{\hat{Y}}^{(i)} = \frac{\sum_{j=1}^M \hat{y}^{(i)}_j}{M} \\
  \sigma_{\hat{Y}}^{(i)} = \sqrt{\frac{\sum_{j=1}^M \left(\hat{y}^{(i)}_{j} - \mu_{\hat{Y}}^{(i)}\right)^2}{M}}
  $$


For each datapoint, we can hence construct a confidence interval, e.g., by summing and subtracting 2*std from the mean prediction:
$$
\text{CI}_i = \left[\mu_{\hat{Y}}^{(i)} - 2\sigma_{\hat{Y}}^{(i)}, \mu_{\hat{Y}}^{(i)} + 2\sigma_{\hat{Y}}^{(i)}\right]
$$

Note: we will implement these metrics using NumPy: these can be further adapted to Pytorch or Tensorflow


### Coverage

![](https://drive.google.com/uc?export=view&id=191dtIqeVOyPhLBJXxzItaJwyCQNaIUA1)

Coverage calculates whether the confidence interval contains the ground truth for each of the datapoints within the dataset. It is hence an extension of accuracy for probabilistic models for regression.

$$
\text{coverage}(y,\hat{y}) = \frac{\sum_{i=1}^{N} \mathbb {1}_{\left[ y^{(i)} \in \text{CI}_i   \right]}}{N}
$$

Note: $\mathbb{1}_{[\text{condition}]}$ is the **indicator function**, which returns 1 if the condition in brackets is true, 0 if false:

$$
\mathbb{1}_{[\text{condition}]} =
  \begin{cases}
    1 &\text{if condition true} \\
    0 &\text{otherwise}
  \end{cases}
$$

**Your task**:
you are given `y_true`, the ground truth of a regression model (a NumPy ndarray of size `(N,)`) and `y_pred`, the prediction output of a `StochasticRegressor` model from `keras_uncertainty`.

_Tips_: you can find the weights of a pretrained MCDropout regressor on the GitHub page, under the folder `files`, its name is `mcdropout_regression_weights.keras`. The corresponding architecture is the following:

```python
backbone_dropout = keras.models.Sequential([
    keras.Input(shape=(1,)),
    keras.layers.Dense(units=16, activation="relu"),
    keras_uncertainty.layers.StochasticDropout(dropout_p),
    keras.layers.Dense(units=16, activation="relu"),
    keras_uncertainty.layers.StochasticDropout(dropout_p),
    keras.layers.Dense(units=1)
])
```
In addition, you will also find there the predictions corresponding to the test set used in the previous lab (`mcdropout_regression_test_prediction.npy`).

In [ ]:
x_max = 3.14
x_min_pos = 0.5
n = 100
assert n//2 != 0, f"Only even number of data allowed (current {n})"

x_train_reg = np.concatenate([np.linspace(-x_max, -x_min_pos, num=n//2), np.linspace(x_min_pos, x_max, num=n//2)])
y_test_reg = np.sin(x_train_reg)
x_train_reg += np.random.normal(0, 0.1, x_train_reg.shape)

x_test_max = 7
n_test = 1000
x_test_reg = np.linspace(-x_test_max, x_test_max, 1000)
y_test_reg = np.sin(x_test_reg)
# get predictions here

NB predictions are formulated as such by keras_uncertainty:

$y_{pred} = (\mu_{pred}, \sigma_{pred})$

In [ ]:
def coverage(y_true:np.ndarray, y_pred:tuple(np.ndarray)):
  # your code here
  # confidence intervals
  mu = y_pred[0]
  sigma = y_pred[1]
  ci_lower = mu - 2 * sigma
  ci_upper = mu + 2 * sigma
  within_interval = y_true > ci_lower and y_true < ci_upper
  return (within_interval).mean()


TypeError: 'type' object is not iterable

### Brier Score

The Brier Score is an extension of the mean square error for probability distributions for classification. It requires the ground truth to be expressed as per-class probabilities. Hard labels need to be expressed using one-hot encodings. The corresponding predictions can be computed e.g. by _voting_.

Summing up, by supposing a $C$-way classification problem and a test data point $x^{(i)}$ with corresponding ground truth $y^{(i)}$:

* $y^{(i)} = \begin{bmatrix} y_1^{(i)} & y_2^{(i)} & \dots & y_C^{(i)} \end{bmatrix}, y_c^{(i)} \in [0,1]~\forall c, \sum_c y_c^{(i)} = 1$

* $\hat{y}^{(i)} = \begin{bmatrix} \hat{y}_{11}^{(i)} & \dots & \hat{y}_{1C}^{(i)}  \\ \hat{y}_{21}^{(i)} & \dots & \hat{y}_{2C}^{(i)} \\ \vdots & \ddots & \vdots \\ \hat{y}_{M1}^{(i)} & \dots & \hat{y}_{MC}^{(i)} \end{bmatrix}, \hat{y}_{jc}^{(i)} \in [0,1]~\forall j,c; \sum_c \hat{y}_{jc}^{(i)} = 1~\forall j \in {1,\dots,M}$

  We can summarize the output distribution by taking the mean across the various samples (components of the
  ensemble, number of runs for VI...):

  $$
  \bar{\hat{y}}^{(i)} = \begin{bmatrix}\frac{\sum_{i=1}^{M}\hat{y}_{i1}^{(i)}}{M} & \dots & \frac{\sum_{i=1}^{M}\hat{y}_{iC}^{(i)}}{M} \end{bmatrix}
  $$

We can then evaluate our model by having a sample of $N$ test data points, computing $\bar{\hat{y}}$ for each of them:

$$
\text{BS}(y, \hat{y}) = \frac{\sum_{i=1}^{N}\sum_{c=1}^{C}\left(y_{c}^{(i)}-\bar{\hat{y}}_{jc}^{(i)}\right)^2}{NC}
$$

Note: if we have a binary classification problem, the formulation above simplifies, since $y$ is a 0-1 scalar and $\hat{y}$ a vector of probabilities of assignment to the positive class.

For the implementation, we will need first to consider whether the outputs are passed as a simplex (i.e., a probability distribution) or as logits (i.e., a vector of real unbounded values). We will need to apply softmax to the logits in the latter case.

**Your task**: recover one of yesterday's implementations of BNNs for MNIST classification and obtain the output from `StochasticClassifier` (or `EnsembleClassifier`) for the MNIST test set. Compute the brier score associated with the prediction.

In [ ]:
def brier_score(y_true, y_pred, preds_as_logits=False, axis_classes=2):
  # your code here
  pass



## Calibration

Calibration is a measure of **trust** which concerns the relationship between accuracy and confidence.
We expect accuracy that models have in their predictions to be high when the confidence assigned to these predictions is high; viceversa, low confidence should be related to low accuracy.
The usage of accuracy, in this case, makes the concept of calibration quickly applicable to classification problems.

We use the definition of confidence as the level of probability assigned to the predicted class (NB there are also other definitions, like entropy).
If we consider the predictive posterior $P(y|x)$ as $\bar{\hat{y}}$ which we consideded before, then $\text{confidence}(\hat{y}) = \max_{c\in\{1,\dots,C\}}\left(\bar{\hat{y}}_c\right)$. NB: we are talking about **max**, not **argmax** (which is related to the predicted class instead).



### Reliability plots

We divide the 0-1 range for confidence in $K$ bins (e.g., $[0.0,0.1),[0.1,0.2),\dots,[0.9,1.0]$). We call each of these bins $B_k, k\in\{1,\dots,K\}$.
We assign each of the data points in our test dataset to these bins.
We then retrieve the mean accuracy attained by the model on these points:
$$
\text{Acc}(B_k) = \sum_{i=1}^{N}{\mathbb{1}_{\left[i\in B_k\right]}\frac{\text{Acc}(y_i, \hat{y}_i)}{N_k}},
$$
where $N_k$ indicates the number of elements in bin $B_k$.


A reliability plot can be created as a plot with confidence on the x axis and accuracy on the y axis

![](https://www.researchgate.net/profile/Anand-Avati/publication/321160854/figure/fig2/AS:562618466344960@1511150103061/Reliability-curve-calibration-plot-of-the-model-output-probabilities-on-the-test-set.png)

A perfect calibration should happen when confidence = accuracy (black dotted line).


**Your task:** below is a code for creating reliability plots using NumPy. You will notice that the functions for obtaining accuracy and confidence for probabilistic classifiers is incomplete. Complete these formula taking into account that the `predictions` are the output of a `StochasticClassifier`.
Then, use the `reliability_vector` and the `reliability_plot` functions to plot the reliability plot associated with the data from the MNIST classification example.


In [ ]:
def accuracy_prob_models(predictions, ground_truth):
  '''
  NB be careful at how the ground_truth vector is passed; check if it is a
  one-hot encoded vector or a list of ints.
  In the first case, it is necessary to run also this through the argmax(1)
  so we can recover a list of ints from it.
  '''
  return (predictions.argmax(1) == ground_truth).mean()

def confidence_prob_models(predictions):
  '''
  In this case we consider predictions as the probability value of the
  assignment class.
  '''
  return predictions.max(1)

def confidence_binning(confidence_vector, n_bins=10):
  '''
  Returns the bin index associated to each datapoint and the bin composition
  '''
  bins = np.linspace(1/n_bins, 1, n_bins)
  return np.digitize(confidence_vector, bins), bins

def reliability_vector(predictions, ground_truth, n_bins=10):
  '''
  Given predictions and ground truth, calculates the confidence scores associated
  with the predictions, bins the confidence into n_bins equispaced in the [0,1]
  line, then compute the per-bin accuracy.
  Returns a 1-d array of n_bins elements containing the per-bin accuracy, and an
  array containing the cutoffs of each bin.
  '''
  confidence_scores = confidence_prob_models(predictions)

  bins_composition, bins_cutoffs = confidence_binning(confidence_scores, n_bins)

  mean_accuracy_per_bins = np.full((n_bins,), fill_value=np.nan)
  bin_counts = np.bincount(bins_composition)

  for i in range(n_bins):
    if i > bins_composition.max():
      break
    if bin_counts[i] > 0:
      group_accuracy = accuracy_prob_models(
          predictions[bins_composition==i],
          ground_truth[bins_composition==i]
        )
      mean_accuracy_per_bins[i] = group_accuracy

  return mean_accuracy_per_bins, bins_cutoffs

def reliability_plot(reliability_vector, bins_cutoffs, clear_nans=True):
  bins_delta = bins_cutoffs[1] - bins_cutoffs[0]
  x_axis = bins_cutoffs - bins_delta/2

  if clear_nans:
    x_axis = x_axis[~np.isnan(reliability_vector)]
    reliability_vector = reliability_vector[~np.isnan(reliability_vector)]

  fig, ax = plt.subplots()
  ax.scatter(
      x_axis,
      reliability_vector
  )
  ax.set_xlim((0,1))
  ax.set_ylim((0,1))
  line = mlines.Line2D([0, 1], [0, 1], color='red')
  transform = ax.transAxes
  line.set_transform(transform)
  ax.add_line(line)
  ax.set_xlabel("confidence")
  ax.set_ylabel("accuracy")
  plt.plot(x_axis, reliability_vector)
  plt.show()



## Create and load the model for this part

To see how the model was trained, refer to [this Colab notebook](https://colab.research.google.com/drive/1pP3ngtTIXcKb-e7XO2WEC4a-KEB6m3UZ?usp=sharing)

In [ ]:
download_bayesian_model = True

models_classification = {
    "frequentist": {
        "url": "https://www.zullich.it/misc/fmnist_weights.keras",
        "architecture": lambda : keras.Sequential([
                        keras.layers.Input(shape=(28, 28, 1)),
                        keras.layers.Conv2D(filters=16, kernel_size=7, activation="relu"),
                        keras.layers.BatchNormalization(),
                        keras.layers.Conv2D(filters=32, kernel_size=7, activation="relu"),
                        keras.layers.BatchNormalization(),
                        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu"),
                        keras.layers.BatchNormalization(),
                        keras.layers.Conv2D(filters=128, kernel_size=7, activation="relu"),
                        keras.layers.BatchNormalization(),
                        keras.layers.GlobalAveragePooling2D(),
                        keras.layers.Flatten(),
                        keras.layers.Dense(10, activation="softmax")
        ])
      },
    "bayesian": {
        "url": "https://www.zullich.it/misc/fmnist_weights_b.keras",
        "architecture": lambda : keras.Sequential([
                        keras.layers.Input(shape=(28, 28, 1)),
                        keras.layers.Conv2D(filters=16, kernel_size=7, activation="relu"),
                        ku.layers.StochasticDropout(rate=0.1),
                        keras.layers.BatchNormalization(),
                        keras.layers.Conv2D(filters=32, kernel_size=7, activation="relu"),
                        ku.layers.StochasticDropout(rate=0.1),
                        keras.layers.BatchNormalization(),
                        keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu"),
                        ku.layers.StochasticDropout(rate=0.1),
                        keras.layers.BatchNormalization(),
                        keras.layers.Conv2D(filters=128, kernel_size=7, activation="relu"),
                        ku.layers.StochasticDropout(rate=0.1),
                        keras.layers.BatchNormalization(),
                        keras.layers.GlobalAveragePooling2D(),
                        keras.layers.Flatten(),
                        keras.layers.Dense(10, activation="softmax"),
                    ])
    }
}
model_metadata = models_classification["bayesian" if download_bayesian_model else "frequentist"]

download_weights(model_metadata["url"], "weights.keras")



model_fmnist = model_metadata["architecture"]()
model_fmnist.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model_fmnist.load_weights("weights.keras")


In [ ]:
_, (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_test = np.expand_dims(x_test, axis=-1).astype(np.float32) / 255

y_pred = model_fmnist.predict(x_test)


In [ ]:
acc, bins_cutoffs = reliability_vector(y_pred, y_test, n_bins=10)

In [ ]:
# use MNIST test results here
reliability_plot(
    acc, bins_cutoffs
)


**Q: _Is the above model underconfident or overconfident?_**

### Calibration Error

The calibration error can be easily computed out of the binning procedure carried out for obtaining the reliability plots

$$
\text{CE}(y, \hat{y}) = \sum_{k=1}^{K} \left\vert \text{Acc}(B_k) - \text{Conf}(B_k) \right\vert
$$

*As part of the final project, you are required to implement CE and ECE.*

In [ ]:
def calibration_error(..., ...):
  pass
  #your code here

### Expected Calibration Error

One of the issues with CE is that all bins are equally weighted. It would make more sense to have bins with more data be weighted more than others. ECE compensates for that:

$$
\text{ECE}(y, \hat{y}) = \frac{\sum_{k=1}^{K}N_k\cdot \left\vert \text{Acc}(B_k) - \text{Conf}(B_k) \right\vert}{N}
$$

As defined before, $N_k$ is the number of elements belonging to bin $k$.

In [ ]:
def expected_calibration_error(..., ...):
  pass
  #your code here

### Calibration for regression tasks

While models for classification come with an inherent estimate of confidence, this is not true in the case of regression.
Restricting to regression for 1-d targets, in probabilistic models we have a predictive posterior with a certain _precision_ (i.e., inverse of variance). We can intuitively think of a relationship whereas high precision $\Leftrightarrow$ high confidence.

Specifically, we can build **confidence intervals** for a given prediction. For this, we have to fix a **confidence level** $\alpha$ and derive the corresponding interval around a given point, usually the expected value of the predictive posterior; in the case of a Gaussian, we build the interval around $\mu$.

![](https://d37djvu3ytnwxt.cloudfront.net/assets/courseware/v1/3c2230f2c98f3c61fcb4a4884ba96f84/asset-v1:DelftX+OT.1x+3T2016+type@asset+block/Normal_critical_values.png)
Creation of a confidence interval for a Gaussian distribution (in the image denoted as $\underline{z}$) around its expected value with three confidence levels $\alpha=0.950, 0.990, 0.999$. _Image provided by TU Delft in the ["Note on the interpretation of confidence interval" course](https://ocw.tudelft.nl/course-readings/note-interpretation-confidence-interval/) under the CC-BY-SA License_.

For a Gaussian predictive posterior $Z=P(Y|X)$ with mean $\mu$, the task of determining the confidence interval boils down to determine the bounds $z_l$ and $z_u$ such as

$$
P(z_l \leq \mu \leq z_u) = \alpha
$$

In case the predictive posterior is a continuos Gaussian, the solution is $z_l = z_{\frac{1-\alpha}{2}} = \Phi^{-1}\left(\frac{1-\alpha}{2}\right)\cdot\sigma + \mu$; $\Phi^{-1}$ indicating the inverse cumulative distribution function of the standard Gaussian. Due to the symmetry of the Gaussian distribution, $z_u$ can be quickly computed as $2\mu - z_{l}$.

In this way, a high-confidence interval will tend to be quite wide, while a low-confidence intervall will be thin around the expected value.

Now that we have determined how to calculate the confidence intervals, we need to translate the concept of calibration to regression problem. The reliability plot for the regression problem can be built like this:

Instead of doing confidence binning (like before), we:
* Fix equispaced confidence values, e.g., $0.0, 0.1, \dots ,0.9, 1.0$, call each of them $\alpha_k, k=1,\dots,K$. (NB: if we have a predictive posterior with unbounded support, we might wanna skip the value 1.0)
* $\forall \alpha_k$:
  * $\forall x^{(i)} \in X$ test dataset:
    * Calculate the confidence interval for $\hat{y}$ based on $\alpha_k$
    * Calculate the coverage, accuracy metric for regression, $c_k^{(i)} \doteq \text{Coverage}(y, \hat{y})_{\alpha_k}$
    * Add $c_k^{(i)}$ to the plot

Practical notes:
* Using SciPy (or PyTorch, or Tensorflow) we can quickly compute the inverse CDF of a generic Gaussian. In SciPy, the command is `scipy.stats.normal(μ, σ).ppf(α)`, where μ, σ are the mean and std (NB, not the variance!) and α is the quantile. PPF stands for "Percent Point Function", which is another term for the inverse CDF.

Statistical distributions in SciPy accept arrays as arguments, so `scipy.stats.normal([μ1, μ2], [σ1, σ2]).ppf(α)` will return an array of two scalars, the α-th quantile referred to the Gaussian with mean μ1 and std σ1 and the α-th quantile referred to the Gaussian with mean μ2 and std σ2.

We can additionally get multiple quantiles by passing a column vector as α:
we can define `α = np.array([α1,..., αk])` then call `scipy.stats.normal([μ1, μ2], [σ1, σ2]).ppf(α[:, np.newaxis])` to obtain all the α1,..., αk quantiles for the two Gaussians.
NB: Don't skip the transposition into column vector (`α[:, np.newaxis]`)!


In [ ]:
def reliability_vector_regression(prediction, ground_truth, n_bins=10):
  confidence_cutoffs = np.linspace(0, 1, n_bins+1)[:, np.newaxis] # already prepare the correct shape
  quantiles = (1 - confidence_cutoffs) / 2

  predictions_mean, predictions_std = prediction
  interval_lower_bound = stats.norm(predictions_mean.squeeze(), predictions_std.squeeze()).ppf(quantiles)
  interval_upper_bound = 2 * predictions_mean.squeeze() - interval_lower_bound

  in_interval = (ground_truth >= interval_lower_bound) & (ground_truth <= interval_upper_bound)
  coverage = in_interval.mean(1)

  return coverage, confidence_cutoffs

def reliability_plot_regression(reliability_vector, confidence_cutoffs):
  fig, ax = plt.subplots()
  ax.scatter(
      confidence_cutoffs,
      reliability_vector
  )
  ax.set_xlim((0,1))
  ax.set_ylim((0,1))
  line = mlines.Line2D([0, 1], [0, 1], color='red')
  transform = ax.transAxes
  line.set_transform(transform)
  ax.add_line(line)
  plt.plot(confidence_cutoffs, reliability_vector)
  plt.show()

Let's try it on the toy regression...

Let us recreate the regression example and download the ensemble weights

In [ ]:
def regression_gaussian_nll_loss(variance_tensor, epsilon=1e-8, variance_logits=False):
    """
        Gaussian negative log-likelihood for regression, with variance estimated by the model.
        This function returns a keras regression loss, given a symbolic tensor for the sigma square output of the model.
        The training model should return the mean, while the testing/prediction model should return the mean and variance.
    """
    def nll(y_true, y_pred):
        #if variance_logits:
        #    variance_tensor = K.exp(variance_tensor)

        return 0.5 * K.mean(K.log(variance_tensor + epsilon) + K.square(y_true - y_pred) / (variance_tensor + epsilon))

    return nll

def ensemble_component_initializer():
    inp = keras.Input(shape=(1,))
    x = keras.layers.Dense(32, activation="relu")(inp)
    x = keras.layers.Dense(32, activation="relu")(x)
    mean = keras.layers.Dense(1)(x)
    var = keras.layers.Dense(1, activation="softplus")(x)

    train_model = keras.Model(inp, mean)
    pred_model = keras.Model(inp, [mean, var])

    train_model.compile(loss=regression_gaussian_nll_loss(var), optimizer="adam")

    return train_model, pred_model

n_components = 10
n_epochs = 500

ensemble = ku.models.DeepEnsembleRegressor(ensemble_component_initializer, num_estimators=n_components)

In [ ]:
download_weights("https://www.zullich.it/misc/ensemble.keras.zip", "ensemble.keras.zip")

!unzip ensemble.keras.zip

ensemble.load_weights("ensemble.keras")

In [ ]:
y_pred_ensemble = ensemble.predict(x_test_reg)

In [ ]:
rv, cc = reliability_vector_regression(y_pred_ensemble, y_test_reg)
reliability_plot_regression(rv, cc)

**Final task**: let us subset the predictions and the ground truth to only the units in the train set lying in the interval [-3.14, 3.14]. This should correspond (roughly) to the distribution of the training data. Does something change now?

In [ ]:
mask = np.bitwise_and(x_test_reg>-3.14,x_test_reg<3.14)

In [ ]:
x_test_reg_masked = x_test_reg[mask]
y_pred_ensemble_masked = (y_pred_ensemble[0][mask],y_pred_ensemble[1][mask])

In [ ]:
y_test_reg_masked = y_test_reg[mask]

In [ ]:
rv, cc = reliability_vector_regression(y_pred_ensemble_masked, y_test_reg_masked)
reliability_plot_regression(rv, cc)